# Reconstruct endomterium

This notebook is for...
using this data...

the aim is...
it contains....


In [1]:
import os
import novosparc
import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl

import warnings
warnings.filterwarnings("ignore")

from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'

## the data
downloaded here:
placed here on murphy:

In [2]:
dirname = '/mnt/ramdisk'
fname = os.path.join(dirname,'uterus_histo.h5ad')

In [3]:
# read data
adata = sc.read(fname)

In [4]:
adata.obs

,SampleID,log2p1_count,percent_mito,n_genes,DonorID,BiopsyType,Location,Binary Stage,Stage,Broad cell type,Cell type,CellCycle Phase
4861STDY7387181_AAACCTGAGGCATGGT,4861STDY7387181,13.951467,0.057320,3842,A13,Organ_Donor,endometrium_enriched,Proliferative,proliferative,Stromal,eS,S
4861STDY7387181_AAACCTGAGGGCACTA,4861STDY7387181,13.776227,0.043838,3544,A13,Organ_Donor,endometrium_enriched,Proliferative,proliferative,Stromal,eS,G1
4861STDY7387181_AAACCTGAGTGAAGTT,4861STDY7387181,12.894628,0.070791,2507,A13,Organ_Donor,endometrium_enriched,Proliferative,proliferative,Supporting,PV STEAP4,G1
4861STDY7387181_AAACCTGAGTGGGCTA,4861STDY7387181,12.329235,0.071137,1928,A13,Organ_Donor,endometrium_enriched,Proliferative,proliferative,Stromal,dS,G1
4861STDY7387181_AAACCTGCACCGAAAG,4861STDY7387181,12.171176,0.063977,1824,A13,Organ_Donor,endometrium_enriched,Proliferative,proliferative,Stromal,eS,G1
...,...,...,...,...,...,...,...,...,...,...,...,...
4861STDY7771123_TTTGTCATCCACGCAG,4861STDY7771123,10.534303,0.027665,700,A30,Organ_Donor,endometrium_enriched,Secretory,early-secretory,Supporting,PV STEAP4,G1
4861STDY7771123_TTTGTCATCCACTCCA,4861STDY7771123,10.245553,0.065952,635,A30,Organ_Donor,endometrium_enriched,Secretory,early-secretory,Supporting,PV STEAP4,G1
4861STDY7771123_TTTGTCATCCCATTAT,4861STDY7771123,10.759888,0.052510,855,A30,Organ_Donor,endometrium_enriched,Secretory,early-secretory,Supporting,uSMC,G1
4861STDY7771123_TTTGTCATCGGTCTAA,4861STDY7771123,10.208235,0.016920,580,A30,Organ_Donor,endometrium_enriched,Secretory,early-secretory,Stromal,dS,G1


In [11]:
# get broad cell type
col_ct_broad = "Broad cell type"
# get cell types
col_ct = "Cell type"
genenames = list(adata.var.index)
#genenames[1:5]
ngenes = len(genenames)
types = list(adata.obs[col_ct].unique())

print('Number of genes: %d' % ngenes)
print('Number of cells per cell-type:')
print(adata.obs[col_ct].value_counts())
print()
print('NUmber of cells per cell-type, broad tissue:')
print(adata.obs.groupby([col_ct, col_ct_broad]).size().unstack())

Number of genes: 28614
Number of cells per cell-type:
eS                    9178
uSMC                  7511
Endothelial ACKR1     5494
PV MYH11              3390
dS                    2459
PV STEAP4             2202
Other                 1524
Glandular             1113
Fibroblast C7          869
SOX9                   435
Lymphoid               365
Myeloid                298
Endothelial SEMA3G     277
Ciliated               276
Lumenal                 32
Name: Cell type, dtype: int64

NUmber of cells per cell-type, broad tissue:
Broad cell type     Epithelial  Endothelial  Immune  Supporting  Stromal  \
Cell type                                                                  
SOX9                       435            0       0           0        0   
Lumenal                     32            0       0           0        0   
Glandular                 1113            0       0           0        0   
Ciliated                   276            0       0           0        0   
Lymphoid 

In [12]:
# filtering excluded cells
included = ~(adata.obs[col_ct] == 'Excluded')
adata = adata[included]